Versjon 08.01.2020

# Introduksjon

Denne notebooken er ment som en relativt enkel illustrasjon av analyse av sensordata med maskinlæring. Se slides fra introduksjonen for motivasjon. 

# Setup

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from pathlib import Path
import subprocess


# Data

Vi bruker et sensor-datasett fra UCI Machine Learning, tilgjenglig via Kaggle: https://www.kaggle.com/uciml/human-activity-recognition-with-smartphones. Fra 30 personer ble det samlet målinger fra en smart-telefon mens de utførte dagligdagse oppgaver. Vår oppgave er å predikere hvilken oppgave som ble utført direkte fra sensor-målingene. 

Video som beskriver innsamlingen av data: https://www.youtube.com/watch?v=XOEN9W05_4A

In [1]:
import IPython
IPython.display.IFrame(width="560", height="315", src="https://www.youtube.com/embed/XOEN9W05_4A")


Her er en beskrivelse av datasettet, sakset fra Kaggle:

The Human Activity Recognition database was built from the recordings of 30 study participants performing activities of daily living (ADL) while carrying a waist-mounted smartphone with embedded inertial sensors. The objective is to classify activities into one of the six activities performed.

> **Description of experiment**<br>
The experiments have been carried out with a group of 30 volunteers within an age bracket of 19-48 years. Each person performed six activities (WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING) wearing a smartphone (Samsung Galaxy S II) on the waist. Using its embedded accelerometer and gyroscope, we captured 3-axial linear acceleration and 3-axial angular velocity at a constant rate of 50Hz. The experiments have been video-recorded to label the data manually. The obtained dataset has been randomly partitioned into two sets, where **70% of the volunteers was selected for generating the training data and 30% the test data**.

> The sensor signals (accelerometer and gyroscope) were pre-processed by applying noise filters and then sampled in fixed-width sliding windows of 2.56 sec and 50% overlap (128 readings/window). The sensor acceleration signal, which has gravitational and body motion components, was separated using a Butterworth low-pass filter into body acceleration and gravity. The gravitational force is assumed to have only low frequency components, therefore a filter with 0.3 Hz cutoff frequency was used. From each window, a vector of features was obtained by calculating variables from the time and frequency domain.

> **Attribute information**<br>
> For each record in the dataset the following is provided:

> * Triaxial acceleration from the accelerometer (total acceleration) and the estimated body acceleration.
* Triaxial Angular velocity from the gyroscope.
* A 561-feature vector with time and frequency domain variables.
* Its activity label.
* An identifier of the subject who carried out the experiment.

## Last inn og utforsk data

Vi har allerede hentet data fra Kaggle. Plassert i katalogen `../data`

In [ ]:
DATA = Path('../data/sensor')

In [ ]:
train = pd.read_csv(DATA/'train.csv')
test = pd.read_csv(DATA/'test.csv')

Vi har fått to dataframes bestående av en lang rekke sensormålinger, markert med tilhørende aktiviteter:

In [ ]:
train.info()

In [ ]:
test.info()

Vi ser at det er 7352 treningsdata og 2947 testdata.

Hvordan ser data ut? 

In [ ]:
# For å vise alle søylene i data frames:
pd.set_option('display.max_columns', 600)

In [ ]:
train.head()

In [ ]:
test.head()

Her er noen labels:

In [ ]:
np.random.choice(train['Activity'], size=50)

De seks ulike aktivitetene vi skal detektere er:

In [ ]:
np.unique(train['Activity'])

Fordelingen av disse i treningsdata er:

In [ ]:
train['Activity'].value_counts().plot(kind='bar')
plt.show()

## Ekstra: korrelasjoner

Det er helt sikkert stor korrelasjon mellom mange av features i dette datasettet (akselerasjon og gyroskop-features, for eksempel). Vi kan avdekke dette ved å bruke korrelasjonsmatrisen, og så trekke ut egenskapene som er mest korrelert:

In [ ]:
correlation_matrix = train.corr()
correlation_matrix.info()

Korrelasjonsmatrisen er en 562x562-matrise (alle numeriske features korrelert med alle numeriske features). Her er de føste 10 søyler og 10 rader:

In [ ]:
correlation_matrix.iloc[0:10, 0:10]

Vi ønsker å plukke ut parene av features som har høyest korrelasjon. Vi kan gjøre dette med `unstack`, som gjør alle verdiene i søylen helt til venstre (index-søylen) til søyler:

In [ ]:
correlation_matrix.unstack().shape

In [ ]:
correlation_matrix.unstack()[:5]

Vi får altså 315844 entries

In [ ]:
562*562

Nå kan vi plukke ut de 15 minste og største tallene:

In [ ]:
correlation_matrix.unstack().drop_duplicates().sort_values()[:15]

In [ ]:
correlation_matrix.unstack().drop_duplicates().sort_values()[-15:]

Som vi trodde: det er mange features som er veldig høyt korrelert. 

> En kan (ofte med fordel) fjerne features som er veldig høyt korrelert fra data. Det kan øke ytelsen til modellene. Forsøk gjerne dette her!

# Splitt opp data

Vi deler opp data i input X og output y:

In [ ]:
X_train = train.drop('Activity', axis=1)
y_train = train['Activity']

X_test = test.drop('Activity', axis=1)
y_test = test['Activity']

# Modell

Vi bruker vår venn `RandomForestClassifier`. 

> Senere skal du få vite nøyaktig hvordan denne fungerer!

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)

In [ ]:
rf.fit(X_train, y_train)

Hvor bra accuracy får vi?

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, y_pred)

Over 92%! 

Er dette bra? For å svare på det må vi bruke verktøyene vi har lært om for evaluering av klassifikatorer:

# Evaluer resultatet

## Forvirringsmatrise

In [ ]:
from utils import plot_confusion_matrix, plot_confusion_matrix_with_colorbar

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
plot_confusion_matrix_with_colorbar(cm, classes=np.unique(y_test), figsize=(10,10))

## Feature importance

Her er de 10 features som ble vektet høyest av vår modell:

In [ ]:
importances = rf.feature_importances_
# Find index of those with highest importance, sorted from largest to smallest:
indices = np.argsort(importances)[::-1]
for f in range(10): 
    print(f'{X_test.columns[indices[f]]}: {np.round(importances[indices[f]],2)}')

## Permutation importance

Som kjent er feature importance for random forests en veldig ustabil og lite gunstig måte å måle faktisk viktighet av features (ta en titt tilbake på Lab 0 for mer om dette). Permutation importance gir mer stabile, data-drevne estimat av feature importance.  

**Advarsel:** dette tar litt tid siden vi har såpass mange features-søyler som skal shuffles...

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
#?PermutationImportance

In [ ]:
perm = PermutationImportance(rf, random_state=42)

In [ ]:
perm.fit(X_test, y_test)

In [ ]:
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

**Resultat:**

<img width=30% src="assets/permimportance_sensor.png">

# Fin-tuning

Som diskutert i notebooken fra Lab 1 er det en rekke ting en kan gjøre dersom en ikke er fornøyd med ytelsen til en maskinlæringsmodell. En av disse er å justere på såkalte **hyperparametre** i modellen (dvs parametre som ikke settes under trening, men velges av oss). 

Som vi skal se senere (når vi kommer til hvordan random forests fungerer) er det en rekke hyperparametre i random forests som kan influere ytelsen. 

En mye brukt strategi for å finne gode valg av parametre er å *søke* gjennom et bestemt *grid* av potensielle parameterkombinasjoner. Enten ved å forsøke alle (dette kalles **grid search**) eller ved å forsøke et tilfeldig antall valg (dette kalles **randomized search**). Det finnes også andre, mer avanserte former for *hyperparameteroptimalisering*, for eksempel **bayesian search**. Vi skal se på grid search og randomized search. For bayesiansk søk, se for eksempel `scikit-optimize`: https://scikit-optimize.github.io/#skopt.BayesSearchCV.  

(Det er også mulig å søke gjennom ulike *modeller* i tillegg til deres hyperparemetre, men det skal vi ikke gå inn på her)

Vi forsøker:

## Et mulig parametergrid å søke gjennom

Her er to parametergrid som ofte vil fungere bra for random forest. Nøyaktig hvilke parametre som gir mening å forsøke i en gitt situasjon avhenger blant annet av datasettet en har. For å velge klokt her kreves det en del erfaring, samt forståelse av modellen. 

La oss bare velge noe relativt trygt, og så ikke bry oss om dette er det *beste* valget.

Vi lager to grids, et lite; et stort:

In [ ]:
param_grid_small = {
    
    'max_depth': [5, 10, 15, 20, 30, 100, None],
    'n_estimators': [50, 100, 500, 1000]
    
}


param_grid_large = {
     'bootstrap': [True, False],
     'max_depth': [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10],
     'n_estimators': [50, 100, 500, 1000]
    }

Når vi senere kjører grid search, randomized search og bayesian search, blir disse grids konvertert til en type matriser, og alle kombinasjoner av parametre blir potensielle kandidater. 

Det betyr 7x4 = 28 kombinasjoner for `param_grid_small` og 2x12x3x3x4 = 864 kombinasjoner for `param_grid_large`.

## Grid search

Å søke gjennom absolutt alle kombinasjoner i `param_grid_large` blir for kostbart tidsmessig. Vi bruker derfor `param_grid_small`:

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
rf_gs = GridSearchCV(estimator=rf, param_grid=param_grid_small, cv=3, n_jobs=-1)

In [ ]:
rf_gs.fit(X_train, y_train)

In [ ]:
best_gs_model = rf_gs.best_estimator_

In [ ]:
best_gs_model

In [ ]:
best_gs_model.score(X_test, y_test)

Søket klarte i dette tilfellet ikke å finne en bedre parameterkombinasjon enn den vi allerede hadde. 

## Randomized search

Med randomized search er de vi som bestemmer antall (tilfeldig valgte) kombinasjoner som skal forsøkes. Vi kan derfor tillate oss å bruke `param_grid_large`:

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
rf_rs = RandomizedSearchCV(estimator=rf, param_distributions=param_grid_large, n_iter=50, cv=3, n_jobs=-1, random_state=42)

In [ ]:
rf_rs.fit(X_train, y_train)

In [ ]:
best_rs_model = rf_rs.best_estimator_

In [ ]:
best_rs_model

In [ ]:
best_rs_model.score(X_test, y_test)

Fortsatt ikke vesentlig bedre enn vår første modell. 

Med et større søk (det vil si, `n_iter` satt til et større tall) kan det hende at vi oppdager bedre parametre. Men jo større antall forsøk, jo lenger beregningstid...

# Noen oppgaver

> **Din tur!** Klarer du å lage en modell som kan predikere hvilken person som genererte hver sensormåling? <em>Gi meg din mobiltelefon så skal jeg fortelle deg hvem du er</em>

> **Din tur!** Undersøk hvilke bevegelser som best skiller personer fra hverandre.

# Ekstra

La oss forsøke en annen modell: **logistisk regresjon**:

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
std_sc = StandardScaler()
X_train_std = std_sc.fit_transform(X_train)
X_test_std = std_sc.transform(X_test)

In [ ]:
log_reg = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000, C=0.2, random_state=42)
log_reg.fit(X_train_std, y_train)

In [ ]:
log_reg.score(X_test_std, y_test)

Denne modellen scorer 96%, altså langt bedre enn de random forest-variantene vi forsøkte over. Dette illusterer viktigheten av hensiktsmessig modellvalg, tilpasset data og problemstilling man står ovenfor.

## Forvirringsmatrise

In [ ]:
y_pred_logreg = log_reg.predict(X_test)

In [ ]:
cm_logreg = confusion_matrix(y_test, y_pred_logreg)

plot_confusion_matrix_with_colorbar(cm_logreg, classes=np.unique(y_test), figsize=(10,10))

Denne kan sammenlignes fra vår forvirringsmatrise fra random forest:

In [ ]:
plot_confusion_matrix_with_colorbar(cm, classes=np.unique(y_test), figsize=(10,10))

# Ekstra ekstra

For moro skyld, la oss også forsøke noe kraftigere: en gradient boosting-basert modell.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gb = GradientBoostingClassifier(random_state=42, n_estimators=500)

In [ ]:
gb.fit(X_train, y_train)

In [ ]:
gb.score(X_test, y_test)

Ca. 94%.

## Forvirringsmatrise

In [ ]:
y_pred_gs = gb.predict(X_test)

In [ ]:
cm_gs = confusion_matrix(y_test, y_pred_gs)
fig, ax = plt.subplots(figsize=(12,12))
_ = plot_confusion_matrix(cm_gs, classes=np.unique(y_test), ax=ax)

## Ensembling

En (av mange) mulige måter å forbedre modeller på er å bruke **ensembling**: slå sammen prediksjonene fra flere modeller. Dette er spesielt nyttig dersom man har flere, svært ulike modeller, som hver for seg scorer høyt. Sammen kan de da ofte bli enda bedre (*wisdom of the crowd*).

La oss forsøke med de vi har til nå:

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
eclf = VotingClassifier(estimators=[('rf', best_rs_model), ('logreg', log_reg), ('gnb', gb)], voting='soft')

In [ ]:
eclf.fit(X_train, y_train)

In [ ]:
eclf.score(X_test, y_test)

I vårt tilfelle gav ikke dette en bedre modell enn logistisk regresjon alene.

In [ ]:
y_pred_eclf = eclf.predict(X_test)

In [ ]:
cm_eclf = confusion_matrix(y_test, y_pred_eclf)
fig, ax = plt.subplots(figsize=(12,12))
_ = plot_confusion_matrix(cm_eclf, classes=np.unique(y_test), ax=ax)

In [ ]:
# Merk: For å slippe å trene modellene på nytt når de ensembles kunne vi brukt mlextend 
# sin EnsembleVoteClassifier. Det vil spare mye tid:

#!pip install mlextend
#from mlxtend.classifier import EnsembleVoteClassifier
#import copy
#eclf = EnsembleVoteClassifier(clfs=[best_rs_model, gnb], weights=[1,1], refit=False)